# Merging plots of Compound and Uniswap Governance Protocols

**[Johnnatan Messias](https://johnnatan-messias.github.io/), May 2025**

We analyzed the voting history of the Compound and Uniswap Governance Protocol by gathering data from the Ethereum blockchain. This data, collected through an Archive Node, includes all voting history and transfers related to Compound from March 4th, 2020 (starting at block number 9,601,459) to August 19th, 2024 (up to block number 20,563,000).


In [1]:
import os
import pandas as pd
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
import sys
code_dir = os.path.realpath(os.path.join(os.getcwd(), "..", "src"))

sys.path.append(code_dir)

In [3]:
proposal_quorum = 0.04 * 10e6  # 4% of total supply
protocol_names = ['compound', 'uniswap']
dynamic_chart = True
plot_style = 'png' if not dynamic_chart else None

In [4]:
from plot_utils import get_plotly_layout
from plot_utils import colors
import plotly.graph_objects as go
from plotly import express as px
from utils import load_dataframes
width, height = 850, 450

In [5]:
# Set directory paths
data_dir = os.path.abspath(
    os.path.join(os.getcwd(), "..", "data"))
plots_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "plots"))

# Create directories if they don't exist
os.makedirs(data_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)

In [6]:
def load_dataframes(file_dir):
    df = pd.read_csv(file_dir)
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df

In [ ]:
dfs = dict(compound=dict(), uniswap=dict())
for protocol_name in tqdm(protocol_names, desc="Loading protocols files..."):
    protocol_path = os.path.realpath(os.path.join(data_dir, protocol_name))
    filenames = [filename for filename in os.listdir(
        protocol_path) if filename.endswith(".csv.gz")]
    for filename in filenames:
        # print(f"Loading {protocol_name} {filename} dataframes")
        file_dir = os.path.realpath(os.path.join(
            data_dir, protocol_name, filename))
        df = load_dataframes(file_dir)
        dfs[protocol_name][filename.replace("_df.csv.gz", "")] = df

Loading protocols files...: 100%|██████████| 2/2 [00:23<00:00, 11.74s/it]


## Loading Labels


In [8]:
labels_dir = os.path.join(data_dir, "labels")
with open(os.path.join(labels_dir, "labels.json")) as f:
    labels = json.load(f)
print("Labels loaded: {}".format(len(labels)))

Labels loaded: 29738


## Exploratory Data Analysis


### Basic Statistics


In [9]:
for protocol_name in protocol_names:
    print("Protocol: {}".format(protocol_name.capitalize()))
    print("\tIn this dataset, there are {} unique proposals created by {} unique proposers".format(
        dfs[protocol_name]['proposal_created'].proposalId.nunique(), dfs[protocol_name]['proposal_created'].proposer.nunique()))
    print("\t\tThere are {} proposals executed".format(
        dfs[protocol_name]['proposal_executed'].proposalId.nunique()))
    print("\t\tThere are {} proposals queued".format(
        dfs[protocol_name]['proposals_queued'].proposalId.nunique()))
    print("\t\tThere are {} proposals canceled".format(
        dfs[protocol_name]['proposal_cancelled'].proposalId.nunique()))

    print("\tThere are {} unique voters who have casted {} votes".format(
        dfs[protocol_name]['votes'].voter.nunique(), dfs[protocol_name]['votes'].shape[0]))

    print("\tVotes were cast trough {} unique transactions".format(
        dfs[protocol_name]['votes'].transactionHash.nunique()))

    # Empty voting power
    print("\tThere are {} votes with 0 voting power".format(
        dfs[protocol_name]['votes'].query('votes == 0').shape[0]))

Protocol: Compound
	In this dataset, there are 307 unique proposals created by 45 unique proposers
		There are 238 proposals executed
		There are 246 proposals queued
		There are 30 proposals canceled
	There are 4538 unique voters who have casted 14841 votes
	Votes were cast trough 13926 unique transactions
	There are 2463 votes with 0 voting power
Protocol: Uniswap
	In this dataset, there are 67 unique proposals created by 31 unique proposers
		There are 43 proposals executed
		There are 43 proposals queued
		There are 14 proposals canceled
	There are 20695 unique voters who have casted 51580 votes
	Votes were cast trough 51559 unique transactions
	There are 4168 votes with 0 voting power


In [10]:
# Proposals defeated by the community
for protocol_name in protocol_names:
    print("Protocol: {}".format(protocol_name.capitalize()))
    dfs[protocol_name]['proposals_defeated'] = dfs[protocol_name]['votes_weighted'].query(
        'status == "defeated"').sort_values(by='proposalId', ascending=True)
    print("\tThere are {} proposals defeated by the community".format(
        dfs[protocol_name]['proposals_defeated'].proposalId.nunique()))

Protocol: Compound
	There are 36 proposals defeated by the community
Protocol: Uniswap
	There are 10 proposals defeated by the community


### Proposals


In [11]:
for protocol_name in protocol_names:
    print("Protocol: {}".format(protocol_name.capitalize()))
    dfs[protocol_name]['proposals_created_timestamp'] = dfs[protocol_name]['votes_weighted'][['proposalId', 'blockNumber', 'status', 'timestamp']].rename(
        columns={'timestamp': 'proposal_created_timestamp'}).sort_values(by='proposal_created_timestamp')
    display(((dfs[protocol_name]['proposals_created_timestamp']['proposal_created_timestamp'].diff(
    ).dt.total_seconds().describe())/3600/24).to_frame().T)

Protocol: Compound


,count,mean,std,min,25%,50%,75%,max
proposal_created_timestamp,0.003542,5.130986,5.591237,0.0,0.779931,3.194913,7.625903,31.136551


Protocol: Uniswap


,count,mean,std,min,25%,50%,75%,max
proposal_created_timestamp,0.000764,16.362022,20.799404,0.002257,0.905738,6.478125,26.130486,80.989815


In [12]:
data_compound = dfs['compound']['proposals_created_timestamp'].set_index(
    'proposal_created_timestamp').resample('ME')['proposalId'].count()
data_uniswap = dfs['uniswap']['proposals_created_timestamp'].set_index(
    'proposal_created_timestamp').resample('ME')['proposalId'].count()

fig = go.Figure(layout=get_plotly_layout(width=width, height=height))
fig.add_trace(go.Scatter(x=data_compound.index, y=data_compound, line=dict(
    color=colors['blue'], width=4, dash='solid'), mode='lines+markers', name='Compound', marker_size=7))

fig.add_trace(go.Scatter(x=data_uniswap.index, y=data_uniswap, line=dict(
    color=colors['red'], width=3, dash='dash'), mode='lines+markers', name='Uniswap', marker_size=7))

fig.update_layout(
    xaxis_title='Proposal creation time',
    yaxis_title='Number of monthly proposals',
    legend=dict(xanchor='center', x=0.5, y=1.05, orientation='h'),
    yaxis=dict(tickmode='linear', tick0=0, dtick=5)
)
fig.update_xaxes(
    dtick="M3", tickformat="%b\n%Y")
fig.update_yaxes(range=[-1.8, 27])

file_dir = os.path.join(
    plots_dir, "number_of_proposals_per_month_2.pdf")
fig.write_image(file_dir, width=width, height=height, scale=1)

fig.show(plot_style)

### Votes Cast


In [13]:
# Plot showing the distribution of COMP tokens held by voters during a voting period
fig = go.Figure(layout=get_plotly_layout(width=width, height=height))

print("Compound")
display(dfs['compound']['votes'].votes.describe().to_frame().T)

min_votes = 1e-3
proposals_ids = dfs['compound']['proposals_defeated'].proposalId
data = dfs['compound']['votes'].query(
    "(proposalId not in @proposals_ids) and (votes >= @min_votes)"
).groupby('proposalId').agg({'votes': ['mean', 'median', 'sum']})

fig.add_trace(go.Scatter(x=data.index, y=data[('votes', 'median')], line=dict(
    color=colors['blue'], width=2), mode='lines', name='Compound'))

print("Uniswap")
display(dfs['uniswap']['votes'].votes.describe().to_frame().T)

min_votes = 1e-3
proposals_ids = dfs['uniswap']['proposals_defeated'].proposalId
data = dfs['uniswap']['votes'].query(
    "(proposalId not in @proposals_ids) and (votes >= @min_votes)"
).groupby('proposalId').agg({'votes': ['mean', 'median', 'sum']})

fig.add_trace(go.Scatter(x=data.index, y=data[('votes', 'median')], line=dict(
    color=colors['red'], width=2), mode='lines', name='Uniswap'))

fig.update_layout(yaxis_title="Median number of votes", xaxis_title="Proposal ID",
                  xaxis=dict(tickmode='linear', tick0=0, dtick=20),
                  legend=dict(xanchor='center', x=0.5, y=1.15, orientation='h'))

fig.update_yaxes(type="log")

file_dir = os.path.join(
    plots_dir, "distribution_of_voting_power_by_voter_per_proposal_2.pdf")
fig.write_image(file_dir, width=width, height=height, scale=1)

fig.show(plot_style)

Compound


,count,mean,std,min,25%,50%,75%,max
votes,14841.0,13277.57193,40143.130077,0.0,0.0101,0.147143,39.57343,361006.425111


Uniswap


,count,mean,std,min,25%,50%,75%,max
votes,51580.0,47548.762241,491462.888967,0.0,0.194624,1.0,1.6,1.501965e+07


In [18]:
# Plot showing the distribution of COMP tokens held by voters during a voting period
fig = go.Figure(layout=get_plotly_layout(width=width, height=height))

print("Compound")
display(dfs['compound']['votes'].votes.describe().to_frame().T)

min_votes = 1e-3
proposals_ids = dfs['compound']['proposals_defeated'].proposalId
data = dfs['compound']['votes'].query(
    "(proposalId not in @proposals_ids) and (votes >= @min_votes)"
).groupby('proposalId').agg({'votes': ['mean', 'median', 'sum']})

fig.add_trace(go.Scatter(x=data.index, y=data[('votes', 'sum')], line=dict(
    color=colors['blue'], width=2), mode='markers', name='Compound'))

print("Uniswap")
display(dfs['uniswap']['votes'].votes.describe().to_frame().T)

min_votes = 1e-3
proposals_ids = dfs['uniswap']['proposals_defeated'].proposalId
data = dfs['uniswap']['votes'].query(
    "(proposalId not in @proposals_ids) and (votes >= @min_votes)"
).groupby('proposalId').agg({'votes': ['mean', 'median', 'sum']})

fig.add_trace(go.Scatter(x=data.index, y=data[('votes', 'sum')], line=dict(
    color=colors['red'], width=2), mode='markers', name='Uniswap'))

fig.update_layout(yaxis_title="Total number of votes", xaxis_title="Proposal ID",
                  xaxis=dict(tickmode='linear', tick0=0, dtick=20),
                  legend=dict(xanchor='center', x=0.5, y=1.15, orientation='h'))

fig.update_yaxes(type="log")

file_dir = os.path.join(
    plots_dir, "distribution_of_voting_power_by_voter_per_proposal_sum.pdf")
fig.write_image(file_dir, width=width, height=height, scale=1)

fig.show(plot_style)

Compound


,count,mean,std,min,25%,50%,75%,max
votes,14841.0,13277.57193,40143.130077,0.0,0.0101,0.147143,39.57343,361006.425111


Uniswap


,count,mean,std,min,25%,50%,75%,max
votes,51580.0,47548.762241,491462.888967,0.0,0.194624,1.0,1.6,1.501965e+07


### Proposal Lifecycle


In [15]:
fig = go.Figure(layout=get_plotly_layout(width=width, height=height))

data_compound = dfs['compound']['votes_weighted'].set_index('proposalId').sort_index()[
    ['supporter_in_favor_percentage', 'supporter_against_percentage',
        'supporter_abstain_percentage', 'status']
]
data_compound = data_compound.query(
    'status == "defeated" or status == "executed"')
data_compound['margin'] = data_compound['supporter_in_favor_percentage'] - \
    data_compound['supporter_against_percentage']
data_compound.rename(columns={'supporter_in_favor_percentage': 'in-favor',
                              'supporter_against_percentage': 'against', 'supporter_abstain_percentage': 'abstain'}, inplace=True)


data_uniswap = dfs['uniswap']['votes_weighted'].set_index('proposalId').sort_index()[
    ['supporter_in_favor_percentage', 'supporter_against_percentage',
        'supporter_abstain_percentage', 'status']
]
data_uniswap = data_uniswap.query(
    'status == "defeated" or status == "executed"')
data_uniswap['margin'] = data_uniswap['supporter_in_favor_percentage'] - \
    data_uniswap['supporter_against_percentage']
data_uniswap.rename(columns={'supporter_in_favor_percentage': 'in-favor',
                             'supporter_against_percentage': 'against', 'supporter_abstain_percentage': 'abstain'}, inplace=True)

fig.add_trace(go.Box(x=data_compound.status, y=data_compound['margin'],
              name='Compound', marker_color=colors['blue'], line=dict(width=1.5), whiskerwidth=0.5, fillcolor=colors['white']))

fig.add_trace(go.Box(x=data_uniswap.status, y=data_uniswap['margin'],
              name='Uniswap', marker_color=colors['red'], line=dict(width=1.5), whiskerwidth=0.5, fillcolor=colors['white']))

fig.update_layout(yaxis_title="Margin of victory", xaxis_title="Proposal outcome",
                  legend=dict(xanchor='center', x=0.5, y=1.15, orientation='h'), yaxis_ticksuffix="%",  boxmode='group')

# or "inclusive", or "linear" by default
fig.update_traces(quartilemethod="exclusive")

file_dir = os.path.join(
    plots_dir, "proposal_votes_percentage_2.pdf")
fig.write_image(file_dir, width=width, height=height, scale=1)

fig.show(plot_style)